In [1]:
TEST_DATA_PATH = 'test.csv'
MODEL_PATH = "pytorch_model.bin"
BATCH_SIZE = 4
EPOCHS = 4
MAX_LEN = 150
TRAIN_SIZE = 0.95
VAL_SIZE = 0.1
SEED_VAL = 42

In [3]:
import pandas as pd
import numpy as np
import os
import time
import datetime
import random

# Visualisation libraries
# import matplotlib.pyplot as plt

# Pytorch
import torch
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


#Transformers
from transformers import AlbertTokenizer
from transformers import AlbertForSequenceClassification, AdamW, AlbertConfig
from transformers import get_linear_schedule_with_warmup

In [4]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [24]:
test_df = pd.read_csv(TEST_DATA_PATH)
test_df = test_df[:5]
test_df.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [25]:
possible_labels = test_df.sentiment.unique()
print(possible_labels)

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = int(index)

test_df['label'] = test_df.sentiment.replace(label_dict)
print(label_dict)
test_df.head()

['neutral' 'positive' 'negative']
{'neutral': 0, 'positive': 1, 'negative': 2}


,textID,text,sentiment,label
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,1
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,2
3,01082688c6,happy bday!,positive,1
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,1


In [26]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [27]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2', do_lower_case=True)

In [28]:
print(tokenizer.sep_token, tokenizer.sep_token_id)
print(tokenizer.cls_token, tokenizer.cls_token_id)

[SEP] 3
[CLS] 2


In [29]:
sentences = test_df.text.values
labels = test_df.label.values

In [30]:
# Create sentence and label lists
sentences = test_df.text.values
labels = test_df.label.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = MAX_LEN,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)
  

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=BATCH_SIZE)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
c:\users\rites\miniconda3\envs\tweet_env\lib\site-packages\transformers\tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [31]:
# Load AlertForSequenceClassification, the pretrained ALBERT model with a single 
# linear classification layer on top.

model = AlbertForSequenceClassification.from_pretrained(
    "albert-large-v2", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels =3 , # The number of output labels--2 for binary classification.# You can increase this for multi-class tasks
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
    return_dict = False,
)

Some weights of the model checkpoint at albert-large-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-large-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You s

In [32]:
model.load_state_dict(torch.load(MODEL_PATH,map_location=torch.device('cpu')))

<All keys matched successfully>

In [51]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []
p1 = []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]
  print(outputs)
  logits= torch.softmax(outputs[0],dim=1).cpu().detach().numpy()


  # Move logits and labels to CPU
#   logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 5 test sentences...
(tensor([[ 3.9308, -2.1119, -1.8943],
        [-2.0290, -2.8359,  3.8152],
        [-1.4530,  3.3934, -2.3779],
        [-2.3170, -2.4643,  3.6082]]),)
(tensor([[-1.7420, -2.7234,  3.7330]]),)
    DONE.


In [54]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    print(pred_flat)
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [56]:
flat_accuracy(predictions[0], true_labels[0])

[0 2 1 2]


0.25

In [55]:
accuracy_per_class(predictions[0], true_labels[0])

Class: neutral
Accuracy: 1/1

Class: positive
Accuracy: 0/2

Class: negative
Accuracy: 0/1



In [45]:
predictions[0]

array([[ 3.9308424, -2.1118946, -1.8943166],
       [-2.028988 , -2.8358648,  3.8152103],
       [-1.4529651,  3.3934038, -2.3779216],
       [-2.3170342, -2.4643087,  3.6081612]], dtype=float32)

In [23]:
outputs = np.argmax(predictions, axis=1)
outputs

array([[0, 2, 1],
       [3, 2, 1],
       [0, 2, 1],
       [0, 3, 0]], dtype=int64)

In [80]:
true_labels

[array([0, 1, 2, 1], dtype=int64),
 array([1, 1, 2, 2], dtype=int64),
 array([0, 0, 2, 0], dtype=int64),
 array([0, 2, 2, 2], dtype=int64)]

## Sample output

In [66]:
# tokenizer_1 = AlbertTokenizer.from_pretrained('albert-base-v2')
# model_s = AlbertForSequenceClassification.from_pretrained('albert-base-v2')

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
# outputs = model(**inputs, labels=labels)
# loss = outputs.loss
# logits = outputs.logits